In [1]:
from cassandra.cluster import Cluster
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import  Model
from cassandra.cqlengine.management import sync_table
from cassandra.cqlengine import connection
from cassandra.query import dict_factory
from datetime import datetime, timedelta
import time
import cassandra
import random
import uuid
import math
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import datetime
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
host = 'localhost'
port = '3306'
db_name = 'study_de'
user = 'root'
password = ''
url = 'jdbc:mysql://' + host + ':' + port + '/' + db_name
driver = "com.mysql.cj.jdbc.Driver"

In [3]:
keyspace = 'study_de'
cassandra_login = 'cassandra'
cassandra_password = 'cassandra'
cluster = Cluster()
session = cluster.connect(keyspace)

In [5]:
def get_data_from_job(user,password,host,db_name):
    cnx = mysql.connector.connect(user=user, password=password,
                                         host=host,
                                      database=db_name)
    query = """select id as job_id,campaign_id , group_id , company_id from job"""
    mysql_data = pd.read_sql(query,cnx)
    return mysql_data

mysql_data = get_data_from_job(user,password,host,db_name)
mysql_data.head()

C:\Users\cungt\AppData\Local\Temp\ipykernel_6256\321299869.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


,job_id,campaign_id,group_id,company_id
0,2,1,10,1
1,3,1,10,1
2,4,1,10,1
3,5,1,10,1
4,6,1,10,1


In [6]:
def get_data_from_publisher(user,password,host,db_name):
    cnx = mysql.connector.connect(user=user, password=password,
                                         host=host,
                                      database=db_name)
    query = """select distinct(id) as publisher_id from master_publisher"""
    mysql_data = pd.read_sql(query,cnx)
    return mysql_data

mysql_data = get_data_from_publisher(user,password,host,db_name)
mysql_data.head()

C:\Users\cungt\AppData\Local\Temp\ipykernel_6256\1796497250.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


,publisher_id
0,1
1,2
2,3
3,4
4,5


In [ ]:
def generating_dummy_data(n_records,session,user,password,host,db_name):
    publisher = get_data_from_publisher(user,password,host,db_name)
    publisher = publisher['publisher_id'].to_list()
    jobs_data = get_data_from_job(user,password,host,db_name)
    job_list = jobs_data['job_id'].to_list()
    campaign_list = jobs_data['campaign_id'].to_list()
    company_list = jobs_data['company_id'].to_list()
    group_list = jobs_data[jobs_data['group_id']!='']['group_id'].astype(int).to_list()
    i = 0
    fake_records = n_records
    while i <= fake_records:
        create_time = cassandra.util.uuid_from_time(datetime.datetime.now())
        bid = random.randint(0,1)
        interact = ['click','conversion','qualified','unqualified']
        custom_track = random.choices(interact,weights=(70,10,10,10))[0]
        job_id = random.choice(job_list)
        publisher_id = random.choice(publisher)
        group_id = random.choice(group_list)
        campaign_id = random.choice(campaign_list)
        ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        sql = f""" INSERT INTO log_tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('{create_time}',{bid},{campaign_id},'{custom_track}',{group_id},{job_id},{publisher_id},'{ts}')"""
        print(sql)
        session.execute(sql)
        i+=1 
    print("Data Generated Successfully")